In [1]:
import pandas as pd
import numpy as np
import requests
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
%matplotlib inline


## __Gather__

__Collect statistics by Seasons__

In [ ]:
import pandas as pd
import numpy as np
import requests
import sys
from bs4 import BeautifulSoup

def fn_getPlayerInfo(url):
    page1 = requests.get('https://www.basketball-reference.com' + url)
    soup = BeautifulSoup(page1.content, 'html.parser')
    height = soup.find('span', {'itemprop':'height'}).text.replace('-', '.')
    weight = soup.find('span', {'itemprop':'weight'}).text.replace('lb','')
    shoots = 'na'
    country = 'na'
    for t in soup.find_all('p'):
        if 'Shoots' in t.text:
            shoots = t.text.split(':')[2].split('\n')[-2].strip()
        if 'Born' in t.text:        
            country = t.text.split('\n')[-2]

    return height, weight, shoots, country

def fn_getStatsForSeason(url):
    # Request url
    page = requests.get(url)
    # Create soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # Find a list of players
    players_tab = soup.find_all('td', {"data-stat":"player"})
    # Prepare a DataFrame for list of players
    players = []
    tot = len(players_tab)
    i = 1
    for pl in players_tab:
        player = {}
        player['name'] = pl.text
        url = pl.find('a')['href']
        player['url'] = url
        height, weight, shoots, country = fn_getPlayerInfo(url)
        player['height'] = height
        player['weight'] = weight
        player['shoots'] = shoots
        player['country'] = country
        pt = pl.parent
        stats = ['pos','age','team_id','g', 'gs', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'fg2', 'fg2a', 'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
        for stat in stats:
            player[stat] = pt.find('td', {'data-stat':stat}).text
        players.append(player)
        print('Player ', i, '/', tot)
        i = i + 1
    players_df = pd.DataFrame(players)  

    return players_df

SEASON=sys.argv[1]
print('Collecting season ', SEASON)
url = 'https://www.basketball-reference.com/leagues/NBA_' + str(SEASON) + '_totals.html'
df = fn_getStatsForSeason(url)
df['season'] = SEASON
df.to_csv(SEASON + '.csv', index=False, encoding='utf-8-sig')



### Read CSV for each season and consolidate in a single dataframe

In [2]:
seasons_df = pd.DataFrame()
tot_rows = 0
for i in range(1990,2021,1):
    tmp = pd.read_csv('sheets/' + str(i) + '.csv', dtype={'height':'string'})
    print(tmp.shape[0] , ' rows for season ', i)
    tot_rows += tmp.shape[0]
    seasons_df = pd.concat([seasons_df, tmp], ignore_index=True)
print('Calculated total of rows ', tot_rows)    
print('Rows in dataframe ' , seasons_df.shape[0])

459  rows for season  1990
441  rows for season  1991
458  rows for season  1992
449  rows for season  1993
481  rows for season  1994
452  rows for season  1995
545  rows for season  1996
574  rows for season  1997
547  rows for season  1998
507  rows for season  1999
496  rows for season  2000
537  rows for season  2001
500  rows for season  2002
483  rows for season  2003
585  rows for season  2004
585  rows for season  2005
563  rows for season  2006
516  rows for season  2007
595  rows for season  2008
582  rows for season  2009
578  rows for season  2010
625  rows for season  2011
551  rows for season  2012
573  rows for season  2013
611  rows for season  2014
651  rows for season  2015
578  rows for season  2016
595  rows for season  2017
664  rows for season  2018
708  rows for season  2019
624  rows for season  2020
Calculated total of rows  17113
Rows in dataframe  17113


In [3]:
# Backup after gathering
seasons_df.to_csv('sheets/seasons.csv', index=False, encoding='utf-8-sig')
seasons_clean = seasons_df

In [4]:
seasons_df.columns

Index(['name', 'url', 'height', 'weight', 'shoots', 'country', 'pos', 'age',
       'team_id', 'g', 'gs', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'fg2', 'fg2a',
       'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'season'],
      dtype='object')

__Collect information for country codes__

In [ ]:
url = 'https://en.wikipedia.org/wiki/ISO_3166-1'
# Request url
page = requests.get(url)
# Create soup object
soup = BeautifulSoup(page.content, 'html.parser')

country_tab = soup.find_all('table', class_="wikitable sortable")[1]

country_list = country_tab.find_all('tr')

countries = []
for i in range(1,len(country_list)):
    country = country_list[i]
    tds = country.find_all('td')
    country_flag_url = tds[0].find('img')['src']
    country_alpha2 = tds[1].text
    country_name = tds[0].text.strip()
    country_alpha3 = tds[2].text.strip()
    countries.append({'country_alpha2': country_alpha2, 'country_alpha3': country_alpha3, 'country_name': country_name, 'country_flag_url': country_flag_url})
countries_df = pd.DataFrame(countries)    
countries_df.to_csv('sheets/country_codes.csv', index=False, encoding='utf-8-sig')    


## __Assess__

In [5]:
seasons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17113 entries, 0 to 17112
Data columns (total 29 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     17113 non-null  object
 1   url      17113 non-null  object
 2   height   17113 non-null  string
 3   weight   17113 non-null  int64 
 4   shoots   17113 non-null  object
 5   country  17113 non-null  object
 6   pos      17113 non-null  object
 7   age      17113 non-null  int64 
 8   team_id  17113 non-null  object
 9   g        17113 non-null  int64 
 10  gs       17113 non-null  int64 
 11  mp       17113 non-null  int64 
 12  fg       17113 non-null  int64 
 13  fga      17113 non-null  int64 
 14  fg3      17113 non-null  int64 
 15  fg3a     17113 non-null  int64 
 16  fg2      17113 non-null  int64 
 17  fg2a     17113 non-null  int64 
 18  ft       17113 non-null  int64 
 19  fta      17113 non-null  int64 
 20  orb      17113 non-null  int64 
 21  drb      17113 non-null  int64 
 22

In [6]:
seasons_df.describe()

,weight,age,g,gs,mp,fg,fga,fg3,fg3a,fg2,...,fta,orb,drb,ast,stl,blk,tov,pf,pts,season
count,17113.000000,17113.00000,17113.000000,17113.000000,17113.000000,17113.000000,17113.000000,17113.000000,17113.000000,17113.000000,...,17113.000000,17113.000000,17113.000000,17113.000000,17113.000000,17113.00000,17113.000000,17113.000000,17113.000000,17113.000000
mean,217.474084,26.86893,47.499795,22.622451,1107.694501,172.011862,378.112371,28.435517,80.396541,143.576345,...,113.131596,53.885292,139.835505,102.585929,35.974931,22.57056,65.520306,99.257348,457.685035,2005.931514
std,27.542382,4.15474,26.470449,27.932164,903.795995,170.713444,363.727564,43.313561,114.924706,151.043039,...,129.959548,61.590771,138.807254,130.217502,35.423861,33.79051,62.352744,75.869214,459.996682,8.894523
min,133.000000,18.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1990.000000
25%,195.000000,24.00000,24.000000,0.000000,281.000000,32.000000,78.000000,0.000000,2.000000,25.000000,...,20.000000,10.000000,30.000000,15.000000,8.000000,3.00000,15.000000,31.000000,84.000000,1998.000000
50%,218.000000,26.00000,51.000000,7.000000,918.000000,118.000000,268.000000,6.000000,22.000000,93.000000,...,67.000000,31.000000,101.000000,56.000000,26.000000,11.00000,48.000000,89.000000,311.000000,2006.000000
75%,237.000000,30.00000,73.000000,42.000000,1796.000000,265.000000,586.000000,42.000000,122.000000,214.000000,...,162.000000,76.000000,205.000000,138.000000,54.000000,27.00000,99.000000,155.000000,704.000000,2014.000000
max,360.000000,44.00000,85.000000,83.000000,3533.000000,1034.000000,2173.000000,402.000000,1028.000000,961.000000,...,972.000000,523.000000,1007.000000,1164.000000,246.000000,376.00000,464.000000,371.000000,2832.000000,2020.000000


In [7]:
seasons_df.country.unique()

array(['us', 'de', 'pa', 'ss', 'no', 'gb', 'rs', 'jm', 'do', 'lb', 'lt',
       'ng', 'pr', 'me', 'hr', 'ht', 'nl', 'ca', 'bs', 'ru', 'fr', 'eg',
       'tt', 'au', 'cd', 'br', 'lv', 'cu', 'ro', 'sk', 'vi', 'ba', 'it',
       'cz', 'mx', 'ee', 'za', 'ua', 'vc', 'si', 'hu', 'nz', 'sn', 'se',
       'ge', 'gy', 'es', 'dm', 'fi', 'ml', 'ar', 'tr', 'cn', 'cm', 'gr',
       'lu', 'be', 've', 'ie', 'pl', 'gp', 'kr', 'jp', 'uy', 'mq', 'ch',
       'ga', 'tw', 'ir', 'il', 'cg', 'tz', 'gf', 'mk', 'tn', 'cv', 'gh',
       'at', 'lc', 'gn', 'ao', 'sd', 'ag'], dtype=object)

In [8]:
seasons_df.shoots.unique()

array(['Right', 'Left', 'Left Right'], dtype=object)

In [9]:
seasons_df.query('name == "Trevor Ariza" and season == 2020')

,name,url,height,weight,shoots,country,pos,age,team_id,g,...,fta,orb,drb,ast,stl,blk,tov,pf,pts,season
16506,Trevor Ariza,/players/a/arizatr01.html,6.8,215,Right,us,SF,34,TOT,53,...,74,34,212,91,69,15,56,111,423,2020
16507,Trevor Ariza,/players/a/arizatr01.html,6.8,215,Right,us,SF,34,SAC,32,...,27,21,125,50,35,7,28,63,192,2020
16508,Trevor Ariza,/players/a/arizatr01.html,6.8,215,Right,us,SF,34,POR,21,...,47,13,87,41,34,8,28,48,231,2020


In [10]:
seasons_df.query('name == "Michael Adams"')

,name,url,height,weight,shoots,country,pos,age,team_id,g,...,fta,orb,drb,ast,stl,blk,tov,pf,pts,season
1,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,27,DEN,79,...,314,49,176,495,121,3,141,133,1221,1990
462,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,28,DEN,66,...,529,58,198,693,147,6,240,162,1752,1991
903,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,29,WSB,78,...,360,58,252,594,145,9,212,162,1408,1992
1365,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,30,WSB,70,...,277,52,188,526,100,4,175,146,1035,1993
1809,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,31,WSB,70,...,270,37,146,480,96,6,167,140,849,1994
2292,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,32,CHH,29,...,30,6,23,95,23,1,26,41,188,1995
2741,Michael Adams,/players/a/adamsmi01.html,5.10,162,Right,us,PG,33,CHH,21,...,35,5,17,67,21,4,25,25,114,1996


In [11]:
seasons_df.iloc[1]

name                   Michael Adams
url        /players/a/adamsmi01.html
height                          5.10
weight                           162
shoots                         Right
country                           us
pos                               PG
age                               27
team_id                          DEN
g                                 79
gs                                74
mp                              2690
fg                               398
fga                              989
fg3                              158
fg3a                             432
fg2                              240
fg2a                             557
ft                               267
fta                              314
orb                               49
drb                              176
ast                              495
stl                              121
blk                                3
tov                              141
pf                               133
p

### Summary

__Quality Issues__

    - Missing stats for fg_pct, fg3_pct, fg2_pct, ft_pct
    - Height in string
    - Globalization
    
__Tidiness Issues__

    - Country in codes instead of names
  

## __Clean__

__Missing stats for fg_pct, fg3_pct, fg2_pct, ft_pct__

Calculate the stats

In [12]:
seasons_clean['fg_pct'] = seasons_clean.apply(lambda x: x['fg'] / x['fga'] if x['fga'] > 0 else 0, axis=1)
seasons_clean['fg3_pct'] = seasons_clean.apply(lambda x: x['fg3'] / x['fg3a'] if x['fg3a'] > 0 else 0, axis=1)
seasons_clean['fg2_pct'] = seasons_clean.apply(lambda x: x['fg2'] / x['fg2a'] if x['fg2a'] > 0 else 0, axis=1)
seasons_clean['ft_pct'] = seasons_clean.apply(lambda x: x['ft'] / x['fta'] if x['fta'] > 0 else 0, axis=1)
seasons_clean['efg'] = seasons_clean.apply(lambda x: (x['fg2'] + ( x['fg3'] * 1.5) ) / x['fga'] if x['fga'] > 0 else 0, axis=1)
seasons_clean['pts_g'] = seasons_clean.apply(lambda x: x['pts'] / x['g'] if x['g'] > 0 else 0, axis=1)

In [13]:
seasons_clean[seasons_clean.season == 1990].iloc[1]

name                   Michael Adams
url        /players/a/adamsmi01.html
height                          5.10
weight                           162
shoots                         Right
country                           us
pos                               PG
age                               27
team_id                          DEN
g                                 79
gs                                74
mp                              2690
fg                               398
fga                              989
fg3                              158
fg3a                             432
fg2                              240
fg2a                             557
ft                               267
fta                              314
orb                               49
drb                              176
ast                              495
stl                              121
blk                                3
tov                              141
pf                               133
p

__Height in string__

Convert to meters

In [14]:
seasons_clean['height_m'] = seasons_clean.apply(lambda x: round((int(x['height'].split('.')[0]) * 12 + int(x['height'].split('.')[1])) * 0.0254, 2) , axis=1)


In [15]:
seasons_clean.iloc[0]

name                       Mark Acres
url         /players/a/acresma01.html
height                           6.11
weight                            220
shoots                          Right
country                            us
pos                                 C
age                                27
team_id                           ORL
g                                  80
gs                                 50
mp                               1691
fg                                138
fga                               285
fg3                                 3
fg3a                                4
fg2                               135
fg2a                              281
ft                                 83
fta                               120
orb                               154
drb                               277
ast                                67
stl                                36
blk                                25
tov                                70
pf          

__Globalization__

Classify players as American or Foreigners

In [16]:
seasons_clean['american'] = seasons_clean.apply(lambda x: 'Yes' if x['country'] == 'us' else 'No', axis=1)

In [17]:
seasons_clean.american.value_counts()

Yes    14575
No      2538
Name: american, dtype: int64

__Country in codes instead of names__

Merge seasons and countries spreadsheet

In [18]:
country_codes = pd.read_csv('sheets/country_codes.csv')

In [19]:
seasons_clean['country_code'] = seasons_clean['country'].str.upper()

In [20]:
seasons_clean = pd.merge(seasons_clean, country_codes, how='left', left_on='country_code', right_on='country_alpha2')

In [21]:
seasons_clean.country_name.value_counts()

United States of America    14575
Canada                        159
France                        149
Germany                       113
Spain                         111
                            ...  
Angola                          1
Norway                          1
Ghana                           1
Guinea                          1
Antigua and Barbuda             1
Name: country_name, Length: 83, dtype: int64

In [22]:
seasons_clean.to_csv('seasons_clean.csv', index=False, encoding='utf-8-sig')